In [1]:
from game import Game
import time
import os
import re

## Load maps

In [2]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [3]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [4]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [5]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [6]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [7]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [8]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [9]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

### BFS

In [10]:
# # TODO: Must return moves, number of visited states
from collections import deque

class MyState:
    def __init__(self, player_pos, boxs_pos):
        self.player_pos = tuple(player_pos)
        self.boxs_pos = tuple(boxs_pos)
    def __hash__(self):
        return hash((self.player_pos, self.boxs_pos))

    def __eq__(self, other):
        return self.player_pos == other.player_pos and self.boxs_pos == other.boxs_pos

def is_corner(game, box_index):
    box_locs = game.get_box_locations()
    box_loc = box_locs[box_index]
    x, y = box_loc[0], box_loc[1]
    if ((game.is_wall((x - 1, y)) and game.is_wall((x, y - 1))) or \
       (game.is_wall((x - 1, y)) and game.is_wall((x, y + 1))) or \
       (game.is_wall((x + 1, y)) and game.is_wall((x, y - 1))) or \
       (game.is_wall((x + 1, y)) and game.is_wall((x, y + 1)))):
            return True
    return False

def are_boxs_in_corner(game):
    box_locs = game.get_box_locations()
    goal_locs = game.get_goal_locations()
    for i,box_loc in enumerate(box_locs):
        if is_corner(game,i):
            if box_loc != goal_locs[i]:
                return True
    return False

def solver_bfs(game_map):
    game = Game(game_map)
    visited = set()
    parent = dict()
    queue = deque()

    start_state = MyState(game.get_player_position(), game.get_box_locations())
    queue.append(start_state)
    visited.add((start_state.player_pos, start_state.boxs_pos))
    parent[(start_state.player_pos, start_state.boxs_pos)] = None

    def bfs():
        while queue:
            curr_state = queue.popleft()
            game.set_player_position(curr_state.player_pos)
            game.set_box_positions(list(curr_state.boxs_pos))

            if game.is_game_won():
                return curr_state

            for direction in ['L', 'R', 'U', 'D']:
                prev_player = game.get_player_position()
                prev_boxes = game.get_box_locations()

                if game.apply_move(direction):
                    new_state = MyState(game.get_player_position(), game.get_box_locations())

                    new_state_tuple = (new_state.player_pos, new_state.boxs_pos)
                    if new_state_tuple not in visited and not are_boxs_in_corner(game):
                        visited.add(new_state_tuple)
                        parent[new_state_tuple] = (curr_state, direction)
                        queue.append(new_state)

                    game.set_player_position(prev_player)
                    game.set_box_positions(prev_boxes)

        return None
    
    final_state = bfs()

    def return_path_visited(final_state):
        if final_state is None:
            return None, len(visited)

        path = []
        state_tuple = (final_state.player_pos, final_state.boxs_pos)
        while parent[state_tuple] is not None:
            prev_state, move = parent[state_tuple]
            path.append(move)
            state_tuple = (prev_state.player_pos, prev_state.boxs_pos)
        path.reverse()

        return path, len(visited)

    return return_path_visited(final_state)

path, num_visited_states = solver_bfs(map)
print("Path:", path)
print("Number of visited states:", num_visited_states)
print("Is Solution Valid?", game.is_solution_valid(path))


Path: ['L', 'L', 'R', 'U', 'D', 'D']
Number of visited states: 26
Is Solution Valid? True


### DFS

### DFS

In [11]:
# TODO: Must return moves, number of visited states
import sys
sys.setrecursionlimit(10000)

class MyState:
    def __init__(self, player_pos, boxs_pos):
        self.player_pos = tuple(player_pos)
        self.boxs_pos = tuple(boxs_pos)
    def __hash__(self):
        return hash((self.player_pos, self.boxs_pos))

    def __eq__(self, other):
        return self.player_pos == other.player_pos and self.boxs_pos == other.boxs_pos


def solver_dfs(game_map):
    game = Game(game_map)
    visited = set()
    curr_path_to_goal = set()
    path = []

    def dfs():
        state = MyState(game.get_player_position(), game.get_box_locations())
        if game.is_game_won():
            return True
        if state in curr_path_to_goal:
            return False
        curr_path_to_goal.add(state)
        if state not in visited:
            visited.add(state)

        for direction in ['U', 'D', 'L', 'R']:
            prev_state = MyState(game.get_player_position(), game.get_box_locations())
            if game.apply_move(direction):
                path.append(direction)
                if dfs():
                    return True
                path.pop()
                game.set_player_position(prev_state.player_pos)
                game.set_box_positions(prev_state.boxs_pos)
        curr_path_to_goal.remove(state)
        return False
    
    if dfs():
        return path, len(visited)
    else:
        return None, len(visited)

path, num_visited_states = solver_dfs(map)
print("Path:", path)
print("Number of visited states:", num_visited_states)
print("Is Solution Valid?",game.is_solution_valid(path))                

Path: ['L', 'U', 'D', 'D', 'U', 'L']
Number of visited states: 7
Is Solution Valid? True


### IDS

In [12]:
# # # TODO: Must return moves, number of visited states

class MyState:
    def __init__(self, player_pos, boxs_pos):
        self.player_pos = tuple(player_pos)
        self.boxs_pos = tuple(boxs_pos)
    def __hash__(self):
        return hash((self.player_pos, self.boxs_pos))

    def __eq__(self, other):
        return self.player_pos == other.player_pos and self.boxs_pos == other.boxs_pos
        

def solver_ids(game_map):
    game = Game(game_map)

    
    def dfs(path,curr_path_to_goal,visited,depth):
        state = MyState(game.get_player_position(), game.get_box_locations())   
        if game.is_game_won():
            return True
        if state in curr_path_to_goal:
            return False
        if state not in visited:
            visited.add(state)
        curr_path_to_goal.add(state)

        if len(path) >= depth:
            curr_path_to_goal.remove(state)
            return False
        
        for direction in ['L', 'R', 'U', 'D']:
            prev_state = MyState(game.get_player_position(), game.get_box_locations())
            if game.apply_move(direction):
                path.append(direction)
                if dfs(path,curr_path_to_goal,visited,depth):
                    return True
                else:
                    path.pop()
                    game.set_player_position(prev_state.player_pos)
                    game.set_box_positions(prev_state.boxs_pos)
        curr_path_to_goal.remove(state)
        return False

    
    curr_path_to_goal = set()
    map_height,map_weidth = game.get_map_size()
    max_dapth = map_height * map_weidth
    visited = set()
    depth = 0
    while depth <max_dapth:
        path = []
        if dfs(path,curr_path_to_goal,visited,depth):
            return path,len(visited)
        else:
            depth += 1
    return None,0
path, num_visited_states = solver_ids(map)
print("Path:", path)
print("Number of visited states:", num_visited_states)
print("Is Solution Valid?", game.is_solution_valid(path))



Path: ['L', 'L', 'R', 'U', 'D', 'D']
Number of visited states: 21
Is Solution Valid? True


### A*

In [13]:
# TODO: Must return moves, number of visited states
import heapq


class MyState:
    def __init__(self, player_pos, boxs_pos):
        self.player_pos = tuple(player_pos)
        self.boxs_pos = tuple(boxs_pos)
    def __hash__(self):
        return hash((self.player_pos, self.boxs_pos))

    def __eq__(self, other):
        return self.player_pos == other.player_pos and self.boxs_pos == other.boxs_pos
    
    def __lt__(self, other):
        return (self.player_pos, self.boxs_pos) < (other.player_pos, other.boxs_pos)

def heuristic_1(game):
    manhattan_distance = 0
    player_loc = game.get_player_position()
    box_locs = game.get_box_locations()
    goal_locs = game.get_goal_locations()
    for i, box_loc in enumerate(box_locs):
        manhattan_distance += abs(player_loc[0] - box_loc[0]) + abs(player_loc[1] - box_loc[1])
        manhattan_distance += abs(box_loc[0] - goal_locs[i][0]) + abs(box_loc[1] - goal_locs[i][1])
    return manhattan_distance

def is_corner(game, box_index):
    box_locs = game.get_box_locations()
    box_loc = box_locs[box_index]
    x, y = box_loc[0], box_loc[1]
    if ((game.is_wall((x - 1, y)) and game.is_wall((x, y - 1))) or \
       (game.is_wall((x - 1, y)) and game.is_wall((x, y + 1))) or \
       (game.is_wall((x + 1, y)) and game.is_wall((x, y - 1))) or \
       (game.is_wall((x + 1, y)) and game.is_wall((x, y + 1)))):
            return True
    return False

def heuristic_2(game):
    manhattan_distance = 0
    player_loc = game.get_player_position()
    box_locs = game.get_box_locations()
    goal_locs = game.get_goal_locations()
    for i, box_loc in enumerate(box_locs):
        manhattan_distance += abs(player_loc[0] - box_loc[0]) + abs(player_loc[1] - box_loc[1])
        manhattan_distance += abs(box_loc[0] - goal_locs[i][0]) + abs(box_loc[1] - goal_locs[i][1])
        if is_corner(game, i) and box_loc != goal_locs[i]:
            manhattan_distance += 100000
    return manhattan_distance


def calculate_distance(loc1, loc2):
    return abs(loc1[0] - loc2[0]) + abs(loc1[1] - loc2[1])


def heuristic_fast(game):
    box_locs = game.get_box_locations()
    goal_locs = game.get_goal_locations()
    portals = game.get_portal_locations()
    manhatan_distance = 0
    

    for i,box in enumerate(box_locs):
        min_distance = calculate_distance(box,goal_locs[i])
        for portal in portals:
            min_distance = min(min_distance, calculate_distance(box,portal[0]) + calculate_distance(portal[1],goal_locs[i]))
            min_distance = min(min_distance, calculate_distance(box,portal[1]) + calculate_distance(portal[0],goal_locs[i]))
        manhatan_distance += min_distance



    return manhatan_distance


def solver_astar(game_map, heuristic_func=heuristic_fast, weight=1):
    game = Game(game_map)
    heap = []
    parent = {}
    visited = set()
    start_state = MyState(game.get_player_position(), game.get_box_locations())
    initial_h = heuristic_func(game)
    heapq.heappush(heap, (initial_h*weight, 0, initial_h*weight, start_state))
    parent[start_state] = None

    while heap:
        current_f, current_g,current_h, curr_state = heapq.heappop(heap)

        if curr_state in visited:
            continue
        visited.add(curr_state)
        
        game.set_player_position(curr_state.player_pos)
        game.set_box_positions(list(curr_state.boxs_pos))

        if game.is_game_won():
            path = []
            state = curr_state
            while parent[state] is not None:
                prev_state, move = parent[state]
                path.append(move)
                state = prev_state
            path.reverse()
            return path, len(visited)


        for direction in ['L', 'R', 'U', 'D']:
            prev_player = game.get_player_position()
            prev_boxes = game.get_box_locations()

            if game.apply_move(direction):
                new_state = MyState(game.get_player_position(), game.get_box_locations())
                move_cost = 1
                new_g = current_g + move_cost
                new_h = heuristic_func(game)
                new_f = new_g + (weight * new_h)

                if new_state not in visited:
                    parent[new_state] = (curr_state, direction)
                    heapq.heappush(heap, (new_f, new_g, new_h, new_state))

                game.set_player_position(prev_player)
                game.set_box_positions(prev_boxes)

    return None, len(visited)

def solver_weighted_astar(game_map, heuristic_func=heuristic_fast, weight=4):
    return solver_astar(game_map,heuristic_func, weight)

path, num_visited_states = solver_astar(map)
print("Path:", path)
print("Number of visited states:", num_visited_states)
print("Is Solution Valid?",game.is_solution_valid(path))

Path: ['L', 'L', 'R', 'D', 'U', 'U']
Number of visited states: 21
Is Solution Valid? True


### Heuristic Functions

In this question, I used three different heuristic functions.

---

#### $\textbf{heuristic\_1}$

This is the most basic function that I wrote, which calculates the Manhattan distance of the player from each box plus the Manhattan distance of each box to its goal.  
This is not a perfect function to use. Although it is admissible, it is far from reality becauseiIt ignores the actual difficulty of accessing boxes(near or far from player), so two very different states can have the same heuristic value.,as the result we may select a bad node for expanding.

Note: If you apply this function for $\textbf{map\_9}$, you won't get the result.

---

#### $\textbf{heuristic\_2}$

In this heuristic, I combined the first heuristic function idea with some deadlock detection because whenever we reach a corner, there are no moves that we can do to move our box from the corner.  
So going to a node where our box goes to the corner and that corner is not the box's goal can be very bad, and by this, we may get stuck in a loop.  
But this function doesnt work well in large maps like mape_9 and still isnt perfent.

Note: As with the above function, if you run this code for $\textbf{map\_9}$, you won't get the result.

---

#### $\textbf{heuristic\_fast}$

As explained above, for $\textbf{map\_9}$, we don't have any solution by now. So I tried to write a very simple and efficient function.  
In this function, I used the Manhattan distance of each box to its goal and added the minimum Manhattan distance of the boxes to the nearest portal.  
With this adjustment,this function gives more weight to the portal's and boxes location, so we may choose better nodes to expand .
Also it's extremely fast and lightweight, making it pretty good for large or complex maps.

Note: We have the solution for map_9 using this heuristic function with weighted A* algorithm.



### Finding best weight for Weighted A*:

By trying different weights I understood that the best weight is 4

## Solve

In [14]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
    "weighted A*" : solver_weighted_astar
}

In [15]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

In [16]:
solve(9, "weighted A*")

weighted A* took 0.86 seconds on map 9 and visited 14980 states.
59 moves were used: ['R', 'U', 'R', 'R', 'U', 'U', 'U', 'L', 'U', 'U', 'U', 'L', 'D', 'L', 'D', 'R', 'R', 'U', 'R', 'D', 'D', 'D', 'D', 'D', 'U', 'R', 'R', 'D', 'D', 'R', 'D', 'L', 'D', 'L', 'U', 'L', 'L', 'U', 'R', 'U', 'R', 'U', 'R', 'D', 'R', 'D', 'L', 'D', 'L', 'L', 'L', 'L', 'U', 'U', 'R', 'R', 'U', 'R', 'D']


## Results:

In [17]:
import pandas as pd
import numpy as np
import time

algorithms = ["BFS", "DFS", "IDS", "A*", "Weighted A*"]
maps = list(np.arange(1, 11))

maps_to_test_plan = {
    1: ["BFS", "DFS", "IDS", "A*", "Weighted A*"],
    2: ["BFS", "DFS", "IDS", "A*", "Weighted A*"],
    3: ["BFS", "DFS", "IDS", "A*", "Weighted A*"],
    4: ["BFS", "DFS", "IDS", "A*", "Weighted A*"],
    5: ["BFS", "IDS", "A*", "Weighted A*"],
    6: ["BFS", "A*", "Weighted A*"],
    7: ["BFS", "A*", "Weighted A*"],
    8: ["BFS", "IDS", "A*", "Weighted A*"],
    9: ["Weighted A*"],
    10: ["BFS", "A*", "Weighted A*"]
}

map_results = {}

for map_number in maps:
    file_name = "map" + str(map_number) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()

    data = []

    for algorithm in algorithms:
        if algorithm in maps_to_test_plan[map_number]:
            start_time = time.time()

            if algorithm == "BFS":
                path, visited = solver_bfs(game_map)
            elif algorithm == "DFS":
                path, visited = solver_dfs(game_map)
            elif algorithm == "IDS":
                path, visited = solver_ids(game_map)
            elif algorithm == "A*":
                path, visited = solver_astar(game_map, heuristic_func=heuristic_fast, weight=1)
            elif algorithm == "Weighted A*":
                path, visited = solver_weighted_astar(game_map, heuristic_func=heuristic_fast, weight=4)

            end_time = time.time()
            runtime = round(end_time - start_time, 2)
            solution = path if path else "No solution"
            move_number = len(path) if path else "No solution"

        else:
            runtime = "-"
            visited = "-"
            solution = "-"
            move_number = "-"

        data.append([runtime, visited, solution, move_number])

    df = pd.DataFrame(data, index=algorithms, columns=["Time", "Visited States", "Solution","Number of Moves"])
    map_results[f"Map_{map_number}"] = df



### Map 1:

In [18]:
print("Map 1")
map_results[f"Map_1"]

Map 1


,Time,Visited States,Solution,Number of Moves
BFS,0.00,47,"[L, R, R, L, U, D, D]",7
DFS,0.00,10,"[U, D, D, U, L, R, R]",7
IDS,0.01,43,"[L, R, R, L, U, D, D]",7
A*,0.00,44,"[L, R, R, L, D, U, U]",7
Weighted A*,0.00,14,"[U, D, L, R, R, L, D]",7


### Map 2:

In [19]:
print("Map 2")
map_results[f"Map_2"]

Map 2


,Time,Visited States,Solution,Number of Moves
BFS,0.0,26,"[L, L, R, U, D, D]",6
DFS,0.0,7,"[L, U, D, D, U, L]",6
IDS,0.0,21,"[L, L, R, U, D, D]",6
A*,0.0,21,"[L, L, R, D, U, U]",6
Weighted A*,0.0,10,"[L, U, D, L, R, D]",6


### Map 3:

In [20]:
print("Map 3")
map_results[f"Map_3"]

Map 3


,Time,Visited States,Solution,Number of Moves
BFS,0.00,103,"[U, L, D, D, U, U, U, U, R, D, D, D, D]",13
DFS,0.00,68,"[U, L, U, U, R, D, U, L, D, D, D, U, U, U, R, ...",33
IDS,0.03,121,"[U, L, D, D, U, U, U, U, R, D, D, D, D]",13
A*,0.01,81,"[U, L, D, D, U, U, U, U, R, D, D, D, D]",13
Weighted A*,0.00,20,"[U, L, D, D, U, U, U, U, R, D, D, D, D]",13


### Map 4:

In [21]:
print("Map 4")
map_results[f"Map_4"]

Map 4


,Time,Visited States,Solution,Number of Moves
BFS,0.0,2,No solution,No solution
DFS,0.0,2,No solution,No solution
IDS,0.0,0,No solution,No solution
A*,0.0,2,No solution,No solution
Weighted A*,0.0,2,No solution,No solution


### Map 5:

In [22]:
print("Map 5")
map_results[f"Map_5"]

Map 5


,Time,Visited States,Solution,Number of Moves
BFS,0.36,6967,"[L, U, L, D, L, R, D, R, D, L, L, U, L, U, U]",15
DFS,-,-,-,-
IDS,8.55,6423,"[L, U, L, D, L, R, D, R, D, L, L, U, L, U, U]",15
A*,0.02,835,"[U, L, D, D, R, D, L, L, L, U, U, U, R, U, L]",15
Weighted A*,0.0,58,"[L, U, L, D, L, R, D, R, D, L, L, U, L, U, U]",15


### Map 6:

In [23]:
print("Map 6")
map_results[f"Map_6"]

Map 6


,Time,Visited States,Solution,Number of Moves
BFS,0.92,23930,"[R, R, U, U, U, U, U, R, L, L, L, L, L, L, L, ...",34
DFS,-,-,-,-
IDS,-,-,-,-
A*,0.38,7036,"[D, D, D, D, D, R, R, R, L, L, L, L, L, L, L, ...",34
Weighted A*,0.06,1327,"[D, D, D, D, D, R, R, R, L, L, L, L, L, L, L, ...",34


### Map 7:

In [24]:
print("Map 7")
map_results[f"Map_7"]

Map 7


,Time,Visited States,Solution,Number of Moves
BFS,4.05,59882,"[R, U, R, R, D, D, D, D, L, D, R, U, U, U, U, ...",34
DFS,-,-,-,-
IDS,-,-,-,-
A*,3.22,47581,"[R, U, R, R, D, D, D, D, L, D, R, U, U, U, U, ...",34
Weighted A*,0.25,2348,"[R, U, R, R, D, L, L, L, R, R, R, D, D, D, L, ...",38


### Map 8:

In [25]:
print("Map 8")
map_results[f"Map_8"]

Map 8


,Time,Visited States,Solution,Number of Moves
BFS,0.72,13097,"[U, U, R, D, L, D, R, R, D, R, U, R, D, R]",14
DFS,-,-,-,-
IDS,12.79,9650,"[U, U, R, D, L, D, R, R, D, R, U, R, D, R]",14
A*,0.01,261,"[U, U, R, D, L, D, R, R, D, R, R, R, D, D, R]",15
Weighted A*,0.01,126,"[U, R, U, R, D, R, D, L, D, R, R, D, R, U, D, ...",22


### Map 9:

In [26]:
print("Map 9")
map_results[f"Map_9"]

Map 9


,Time,Visited States,Solution,Number of Moves
BFS,-,-,-,-
DFS,-,-,-,-
IDS,-,-,-,-
A*,-,-,-,-
Weighted A*,0.81,14980,"[R, U, R, R, U, U, U, L, U, U, U, L, D, L, D, ...",59


### Map 10:

In [27]:
print("Map 10")
map_results[f"Map_10"]

Map 10


,Time,Visited States,Solution,Number of Moves
BFS,5.13,163793,"[R, R, R, R, R, D, R, U, L, U, R, U, L, L, L, ...",46
DFS,-,-,-,-
IDS,-,-,-,-
A*,5.36,68722,"[R, R, R, R, R, D, R, U, L, U, R, U, L, L, L, ...",46
Weighted A*,0.22,4224,"[R, R, R, R, R, D, R, U, L, U, R, U, L, L, L, ...",46


# Questions:

## Q1:

#### $\textbf{State}$

Our state can be considered as the whole map.  
However, we can define it in a better way. Since the positions of the walls and goals remain constant throughout the entire search process, including them in the state uses extra memory unnecessarily, which increases the search time.

A better definition for the state is to store only the player's location and the boxes' locations.  
By doing this, we only save the important and changeable parts of the map, ignoring static elements that never change. As a result, we use less memory and achieve a faster search algorithm.


## Q2:

#### $\textbf{Actions}$

In this problem, actions are the moves the player can make.  
The player can go in four directions:
- Up ($U$)
- Down ($D$)
- Left ($L$)
- Right ($R$)

If the player pushes a box and the next spot is free, the box moves too.(we can only push one box at a time and if we have two box we cant move them)

We also have portals.  
When the player steps on a portal, they instantly move to the connected portal's location.  
Portals only move the player, not the boxes.

Actions are only possible if the player isn't blocked by a wall or trying to push a box into something that can't move.


## Q3:

#### $\textbf{Modeling}$

- **State:** Player position and boxes' positions.
- **Actions:** Up, Down, Left, Right.
- **Initial State:** The starting positions of the player and the boxes.
- **Goal State:** When all boxes are on their assigned goals (this is checked using the `is_game_won()` function).


# Q4:
### Improving the Search Algorithm

To make the search faster and better, I have some ideas:

- We can use a **visited set**. Every time we visit a node, we add it to the set so we don’t visit the same state again. This helps avoid loops and cycles.

- Another good idea is to use **heuristic functions**, like in **A\*** and **Weighted A\***. Heuristics help us choose better moves instead of checking every possible option.

- With the help of the visited set and heuristics, we don’t have to expand all nodes. We focus on better ones, like:
  - Nodes we haven’t visited yet.
  - Nodes where the boxes aren’t stuck in corners (unless the goal is in the corner).

- Also, having a **good state definition** helps a lot. Like we said before, if we only keep track of the player's position and the boxes' positions, it saves memory and speeds things up.

#### Other things that can help:
- **Check for deadlocks** so we don’t waste time on impossible states.(this can be defined as a heuristic function)
- **Check if the goal is reached** after every move to stop early if we're done.(This is important in writing fast functions)
- Try smarter moves first, like pushing boxes closer to their goals.(this can be the heuristic function to)


# Q5:

### Explaining the Algorithms:

- **BFS :**  
  In this algorithm, when we reach a node, the next nodes to be expanded are its neighbors, as long as they haven't been visited before.  
  BFS **always finds the optimal solution** because it searches level by level (depth by depth).  
  If the goal is reachable from multiple paths, BFS guarantees finding the shortest path to the goal.
  Also BFS is a complete algoritm and despite DFS will never get stuck in loops.

  **Problem:** 
    - **Memory usage is very high.**  
    - In large graphs, BFS requires a lot of memory and time to find a solution.
    - For example, in **map_9**, BFS cannot find the solution efficiently.  
    (If you run the code for about **7 minutes**, you still won’t get an answer.)
    So BFS is pretty good when we have small maps.

- **DFS:**  
  DFS goes **deep into one path** before backtracking.  
  It **uses much less memory than BFS** and can sometimes find a solution faster.  

  **Problem:**  
  - **Not optimal**  (It does not guarantee the shortest path. ) 
  - **Can get stuck in loops**(to avoid this we use a visited set for each path so we wont stuck in a loop)  
  - **Can explore deep, useless paths** and miss better solutions.  
  - Works well when we just need **any solution**, not necessarily the best one.
  (like BFS it doesnt work for map_9 and also it doesnt work for some other maps) 
  if you compare the BFS and DFS result it is obvious that DFS it is not always optimal and in some cases can be far away from the best solution.

- **IDS:**  
  IDS is an algorithm that combines both BFS and DFS.  
  It runs **DFS with increasing depth limits**, meaning it **searches layer by layer like BFS** but **avoids storing too many nodes** at once.  
  As a result, it **uses less memory** but is **slower** compared to BFS because it **explores shallow depths multiple times**.  

  Since IDS searches depth by depth, the solution it finds will be **optimal**, as the shortest path exists in the lowest depth level, and IDS ensures it finds it first.

  **Problems:**
  - IDS is **slower than BFS**, so for some maps, it takes a long time to reach the goal.  
    _(For example, after **map 5**, it takes several minutes to finish.)_
  - If there is **no depth limit**, and the map has **no solution**, IDS will **run forever** until it reaches the maximum recursion depth, causing a **maximum recursion call**.

**A***:  
A* is a **smart search algorithm** that doesn’t just explore blindly like BFS or DFS. Instead, it **picks the move that seems best** at each step.  
You can think of it like **having a brain**—it doesn’t just go in every direction but **chooses the one that looks like it leads to the goal faster**.

This "brain" is the **heuristic function**. The **better the heuristic**, the **faster** A* finds a solution.So by having a good heuristic function we have a very fast algorithm.

**Problems:**  
- **A* doesn’t always find the best path** (It is not always optimal)because it picks nodes based on:  
  \[
  f(n) = h(n) + g(n)
  \]
- If the **heuristic isn’t great**, A* can **make bad choices** and take a longer path.

**Weighted A:***

 This is like the A* algorithm but the formula is like this:
   \[
  f(n) = w*h(n) + g(n)
  \]
  it is faster than the regular A* because it grow faster but we may loose optimality.
  Note:This is the only algorithm that works for **maps_9**

- **part a):**

The big problem with **DFS** is that it **doesn't always find the best solution**. You can see this by checking the results for different maps, sometimes it picks a much worse path than BFS.  

Another issue is that **DFS can go really deep and get stuck**. That means it **takes too long** and might **never find the goal** in some cases.  
For example, in **map 5**, if you run DFS for **2-3 minutes**, you **still won’t get an answer**. But if you use **IDS**, it **finds the goal in less than 10 seconds**. DFS just keeps going deep, while IDS **limits the depth** and avoids getting stuck.  

DFS also has a problem with **cycles**—if you don’t handle them, it might **keep looping forever**.  

- Why is DFS still useful?  
- **Takes way less memory** than BFS.  
- **Good if we just need any solution**, not the best one.  

- **part b)**
 The answer is in the explanation of the IDS algorithm.






# Q6

 The result of each map was in a data frame and it was calculated before and it matches our explanation for different alggorithm such that: 
- BFS takes a lot of memory(visited nodes number)
- DFS can stuck in long depth and it doesnt always find the optimal answer
- IDS always find the optimal answer but uses less memory than BFS (comparing the number of visited nodes) but it can be time cosuming because we visit the shorter depth nodes many time so in time comprasion BFS is better (for example if you run this algorithm for map 6,7,9,10 it doesnt find the answer in 2-3 minutes)
- A* is a fast algorithm but doesnt always find the optimal answer (this can be understoode by comparing the BFS results and the A* result)
- Weighted A* can be faster than the A* algorithm if you find a good heuristic function and a good weight by this things you can have a good solution but like the A* this algorithm doesnt always find the optimal solution.

- Note: From the results the only algorithm that works for **maap_9** is the weighted A* with proper heuristic function and weight the other algorithms dont find ythe answer in 5-7 minutes.

# Q7:
 This question was answered before but if I want to talk about is our heuristic admisible and consistent or not I should say that all our heuristic functions are both admisibile and consistent because it use triangeometric relations so this prove that our heuristics are admisible and for consistency I should say that again it is because of the triangeometric relations and we now most of admisibile heurisrics are consistent too if we dont have a random thing so our heuristics are both admisible and consistent so they are proper to use. 

# Q8:

In [28]:
import pandas as pd
import numpy as np
import time

heuristics = {
    "heuristic 1":heuristic_1,
    "heuristic 2": heuristic_2,
    "heuristic fast":heuristic_fast
}

maps = list(np.arange(1,11))

results = []


for map_number in maps:
    file_name = "map" + str(map_number) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    for heuristic_name,heuristic_func in heuristics.items():
        start_time = time.time()
        if map_number == 9:
            if heuristic_name == "heuristic fast":
                path,visited = solver_weighted_astar(game_map,heuristic_func,weight=4)
            else:
                path,visited = "-","-"
        else:
            path, visited = solver_weighted_astar(game_map, heuristic_func=heuristic_func, weight=4)
        end_time = time.time()

        runtime = round(end_time - start_time, 2)
        if map_number == 9:
            if heuristic_name != "heuristic fast":
                runtime = "-"
        solution = "Found" if path else "No solution"
        results.append({
            "Map": f"Map {map_number}",
            "Heuristic": heuristic_name,
            "Time (s)": runtime,
            "Visited States": visited,
            "Path": path
        })

df = pd.DataFrame(results)
df_pivot = df.pivot(index="Map", columns="Heuristic", values=["Time (s)", "Visited States", "Path"])
df_pivot

        

        



Time (s)                            Visited States              \
Heuristic heuristic 1 heuristic 2 heuristic fast    heuristic 1 heuristic 2   
Map                                                                           
Map 1             0.0         0.0            0.0             44          44   
Map 10           0.88        0.64            0.2          20647       13234   
Map 2             0.0         0.0            0.0             18          18   
Map 3             0.0         0.0            0.0             50          45   
Map 4             0.0         0.0            0.0              2           2   
Map 5             0.0         0.0            0.0            143         143   
Map 6            0.29        0.48           0.09           5884        5884   
Map 7            4.91        0.64           0.17          82709        7010   
Map 8             0.0         0.0           0.01             44          44   
Map 9               -           -           1.85              -           -   

                                                                       Path  \
Heuristic heuristic fast                                        heuristic 1   
Map                                                                           
Map 1                 14                              [D, U, R, L, L, R, U]   
Map 10              4224  [R, R, R, R, R, D, R, U, L, U, R, U, U, L, L, ...   
Map 2                 10                                 [L, D, U, L, R, U]   
Map 3                 20         [U, L, U, U, R, D, D, D, D, U, U, L, D, D]   
Map 4                  2                                               None   
Map 5                 58  [L, U, L, D, R, D, L, L, U, R, U, L, D, L, D, ...   
Map 6               1327  [D, D, D, D, D, R, R, R, L, L, L, L, L, L, L, ...   
Map 7               2348  [R, U, R, R, D, D, D, D, L, D, R, L, U, R, U, ...   
Map 8                126  [U, R, R, R, R, R, R, R, R, R, R, R, R, R, U, ...   
Map 9              14980                                                  -   

                                                              \
Heuristic                                        heuristic 2   
Map                                                            
Map 1                                  [D, U, R, L, L, R, U]   
Map 10     [R, R, R, R, R, D, R, U, L, U, R, U, U, L, L, ...   
Map 2                                     [L, D, U, L, R, U]   
Map 3             [U, L, U, U, R, D, D, D, D, U, U, L, D, D]   
Map 4                                                   None   
Map 5      [L, U, L, D, R, D, L, L, U, R, U, L, D, L, D, ...   
Map 6      [D, D, D, D, D, R, R, R, L, L, L, L, L, L, L, ...   
Map 7      [R, U, R, R, D, D, D, D, L, D, R, L, U, R, U, ...   
Map 8      [U, R, R, R, R, R, R, R, R, R, R, R, R, R, U, ...   
Map 9                                                      -   

                                                              
Heuristic                                     heuristic fast  
Map                                                           
Map 1                                  [U, D, L, R, R, L, D]  
Map 10     [R, R, R, R, R, D, R, U, L, U, R, U, L, L, L, ...  
Map 2                                     [L, U, D, L, R, D]  
Map 3                [U, L, D, D, U, U, U, U, R, D, D, D, D]  
Map 4                                                   None  
Map 5          [L, U, L, D, L, R, D, R, D, L, L, U, L, U, U]  
Map 6      [D, D, D, D, D, R, R, R, L, L, L, L, L, L, L, ...  
Map 7      [R, U, R, R, D, L, L, L, R, R, R, D, D, D, L, ...  
Map 8      [U, R, U, R, D, R, D, L, D, R, R, D, R, U, D, ...  
Map 9      [R, U, R, R, U, U, U, L, U, U, U, L, D, L, D, ...

from the results above it is obvious that the heuristic fast works better for this game.

# Q9:

In [29]:
import pandas as pd
import numpy as np
import time

algorithms = ["BFS", "DFS", "IDS", "A*", "Weighted A*"]
maps = list(np.arange(1, 11))

maps_to_test_plan = {
    1: ["BFS", "DFS", "IDS", "A*", "Weighted A*"],
    2: ["BFS", "DFS", "IDS", "A*", "Weighted A*"],
    3: ["BFS", "DFS", "IDS", "A*", "Weighted A*"],
    4: ["BFS", "DFS", "IDS", "A*", "Weighted A*"],
    5: ["BFS", "IDS", "A*", "Weighted A*"],
    6: ["BFS", "A*", "Weighted A*"],
    7: ["BFS", "A*", "Weighted A*"],
    8: ["BFS", "IDS", "A*", "Weighted A*"],
    9: ["Weighted A*"],
    10: ["BFS", "A*", "Weighted A*"]
}

map_results = {}
num_test = 5

for map_number in maps:
    file_name = "map" + str(map_number) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()

    data = []

    for algorithm in algorithms:
        paths = []
        visited_counts = []
        times = []
        if algorithm in maps_to_test_plan[map_number]:
            for _ in range(num_test):
                start_time = time.time()

                if algorithm == "BFS":
                    path, visited = solver_bfs(game_map)
                elif algorithm == "DFS":
                    path, visited = solver_dfs(game_map)
                elif algorithm == "IDS":
                    path, visited = solver_ids(game_map)
                elif algorithm == "A*":
                    path, visited = solver_astar(game_map, heuristic_func=heuristic_fast, weight=1)
                elif algorithm == "Weighted A*":
                    path, visited = solver_astar(game_map, heuristic_func=heuristic_fast, weight=4)

                paths.append(path)
                visited_counts.append(visited)
                end_time = time.time()
                runtime = end_time - start_time
                times.append(runtime)
                solution = path if path else "No solution"
                avg_time = round(sum(times)/num_test,3)
                avg_visited = sum(visited_counts) // num_test
                avg_move_num = sum(len(path) for path in paths if path) // num_test
                if avg_move_num == 0:
                    avg_move_num = "No solution"

        else:
            avg_time = "-"
            avg_visited = "-"
            solution = "-"
            avg_move_num = "-"

        data.append([avg_time, avg_visited, solution, avg_move_num])
    
    df = pd.DataFrame(data, index=algorithms, columns=["Average Time", "Avrage Visited States", "Solution","Average Number of Moves"])
    df["Average Time"] = df["Average Time"].astype(str)
    map_results[f"Map_{map_number}"] = df



In [30]:
print("Map 1")
map_results[f"Map_1"]

Map 1


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,0.002,47,"[L, R, R, L, U, D, D]",7
DFS,0.0,10,"[U, D, D, U, L, R, R]",7
IDS,0.005,43,"[L, R, R, L, U, D, D]",7
A*,0.002,44,"[L, R, R, L, D, U, U]",7
Weighted A*,0.0,14,"[U, D, L, R, R, L, D]",7


In [31]:
print("Map 2")
map_results[f"Map_2"]

Map 2


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,0.001,26,"[L, L, R, U, D, D]",6
DFS,0.0,7,"[L, U, D, D, U, L]",6
IDS,0.001,21,"[L, L, R, U, D, D]",6
A*,0.001,21,"[L, L, R, D, U, U]",6
Weighted A*,0.0,10,"[L, U, D, L, R, D]",6


In [32]:
print("Map 3")
map_results[f"Map_3"]

Map 3


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,0.002,103,"[U, L, D, D, U, U, U, U, R, D, D, D, D]",13
DFS,0.003,68,"[U, L, U, U, R, D, U, L, D, D, D, U, U, U, R, ...",33
IDS,0.014,121,"[U, L, D, D, U, U, U, U, R, D, D, D, D]",13
A*,0.002,81,"[U, L, D, D, U, U, U, U, R, D, D, D, D]",13
Weighted A*,0.0,20,"[U, L, D, D, U, U, U, U, R, D, D, D, D]",13


In [33]:
print("Map 4")
map_results[f"Map_4"]

Map 4


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,0.0,2,No solution,No solution
DFS,0.0,2,No solution,No solution
IDS,0.001,0,No solution,No solution
A*,0.0,2,No solution,No solution
Weighted A*,0.0,2,No solution,No solution


In [34]:
print("Map 5")
map_results[f"Map_5"]

Map 5


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,0.148,6967,"[L, U, L, D, L, R, D, R, D, L, L, U, L, U, U]",15
DFS,-,-,-,-
IDS,6.522,6423,"[L, U, L, D, L, R, D, R, D, L, L, U, L, U, U]",15
A*,0.024,835,"[U, L, D, D, R, D, L, L, L, U, U, U, R, U, L]",15
Weighted A*,0.003,58,"[L, U, L, D, L, R, D, R, D, L, L, U, L, U, U]",15


In [35]:
print("Map 6")
map_results[f"Map_6"]

Map 6


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,0.985,23930,"[R, R, U, U, U, U, U, R, L, L, L, L, L, L, L, ...",34
DFS,-,-,-,-
IDS,-,-,-,-
A*,0.29,7036,"[D, D, D, D, D, R, R, R, L, L, L, L, L, L, L, ...",34
Weighted A*,0.044,1327,"[D, D, D, D, D, R, R, R, L, L, L, L, L, L, L, ...",34


In [36]:
print("Map 7")
map_results[f"Map_7"]

Map 7


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,2.85,59882,"[R, U, R, R, D, D, D, D, L, D, R, U, U, U, U, ...",34
DFS,-,-,-,-
IDS,-,-,-,-
A*,2.692,47581,"[R, U, R, R, D, D, D, D, L, D, R, U, U, U, U, ...",34
Weighted A*,0.086,2348,"[R, U, R, R, D, L, L, L, R, R, R, D, D, D, L, ...",38


In [37]:
print("Map 8")
map_results[f"Map_8"]

Map 8


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,0.257,13097,"[U, U, R, D, L, D, R, R, D, R, U, R, D, R]",14
DFS,-,-,-,-
IDS,12.978,9650,"[U, U, R, D, L, D, R, R, D, R, U, R, D, R]",14
A*,0.033,261,"[U, U, R, D, L, D, R, R, D, R, R, R, D, D, R]",15
Weighted A*,0.015,126,"[U, R, U, R, D, R, D, L, D, R, R, D, R, U, D, ...",22


In [38]:
print("Map 9")
map_results[f"Map_9"]

Map 9


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,-,-,-,-
DFS,-,-,-,-
IDS,-,-,-,-
A*,-,-,-,-
Weighted A*,1.107,14980,"[R, U, R, R, U, U, U, L, U, U, U, L, D, L, D, ...",59


In [39]:
print("Map 10")
map_results[f"Map_10"]

Map 10


,Average Time,Avrage Visited States,Solution,Average Number of Moves
BFS,4.843,163793,"[R, R, R, R, R, D, R, U, L, U, R, U, L, L, L, ...",46
DFS,-,-,-,-
IDS,-,-,-,-
A*,4.612,68722,"[R, R, R, R, R, D, R, U, L, U, R, U, L, L, L, ...",46
Weighted A*,0.227,4224,"[R, R, R, R, R, D, R, U, L, U, R, U, L, L, L, ...",46
